In [4]:
from load_data_and_augmentations import *

In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
plt.rcParams['figure.figsize'] = (20,10)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 20}
plt.rc('font', **font)

In [6]:
tensor_transform = get_tensor_transform('Kinetics', True)
train_spat_transform = get_spatial_transform(2)
train_temp_transform = get_temporal_transform()
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

root_dir = '/media/scratch/astamoulakatos/centre_Ch2/'
df = pd.read_csv('../important_csvs/more_balanced_dataset/more_balanced_stratified.csv')
df_train = get_df(df, 50, True, False, False)
class_image_paths, end_idx = get_indices(df_train, root_dir)
train_loader = get_loader(16, 64, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, False, False)
df_valid = get_df(df, 50, False, True, False)
class_image_paths, end_idx = get_indices(df_valid, root_dir)
valid_loader = get_loader(16, 64, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, False)
df_test = get_df(df, 50, False, False, True)
class_image_paths, end_idx = get_indices(df_test, root_dir)
test_loader = get_loader(16, 64, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, False)


In [7]:
end_idx

tensor([    0,   400,   800,   902,  1302,  1477,  1877,  2277,  2677,  3051,
         3305,  3477,  3542,  3942,  4342,  4412,  4574,  4724,  5124,  5262,
         5662,  6062,  6366,  6766,  6920,  7172,  7572,  7634,  8034,  8118,
         8518,  8918,  9318,  9445,  9573,  9973, 10035, 10435, 10560, 10960,
        11266, 11666, 11774, 12174, 12396, 12796, 13196, 13596, 13996, 14396,
        14796, 15196, 15596, 15996, 16396, 16796, 17196, 17596, 17996, 18396,
        18796, 19196, 19596, 19996, 20396, 20796, 21196, 21596, 21996, 22144,
        22292, 22555, 22658, 23058, 23458, 23658, 23995, 24395, 24795, 25078,
        25392, 25792, 26171, 26447, 26847, 27247, 27384, 27560, 27960, 28143,
        28543, 28943, 29267, 29593, 29681, 29752, 29909, 30309, 30630, 31030,
        31430, 31604, 32004, 32404, 32760, 33001, 33226, 33338, 33411, 33811,
        34155, 34278, 34502, 34871, 35006, 35200, 35441, 35630, 35688, 35762,
        36013, 36119, 36519, 36919, 37319, 37717, 38117, 38517, 

In [11]:
sampler = MySampler(end_idx, 16)

In [18]:
seq_length = 16

In [23]:
indices = []
for i in range(len(end_idx) - 1):
    start = end_idx[i]
    end = end_idx[i + 1] - seq_length
    if start > end:
        indices.append(torch.arange(end, start))
    else:
        indices.append(torch.arange(start, end))
indices = torch.cat(indices)
#self.indices = indices

In [24]:
len(indices)

63101

In [25]:
indices = indices[torch.randperm(len(indices))]
t = iter(indices.tolist())

In [27]:
next(t)

27868

In [29]:
indices[0:100]

tensor([27868, 56645,  1846,  8162, 26036, 42055, 11754, 64612, 63860, 10602,
        55474, 26730, 30481, 36482, 40280, 60756,  4849, 16854,  9258,  4672,
        11101, 34779, 59340, 45745, 18416, 52871, 67012, 56829, 43707, 13133,
        47724, 38977, 28007,  4413,  9588, 20360, 19635, 53417, 52585, 31223,
        37144, 23599,  7885, 56083, 66011, 20396, 42225, 18058, 40762, 33364,
        28678, 61501, 29319,  8138, 61864, 61579, 32912, 36830,  5700, 26016,
        60340, 16227, 35634, 58049, 46215, 21289, 11842, 32336, 17022, 35605,
        33710, 65255, 67615, 10988, 33778,  7872, 62022, 64842, 19858, 41500,
        15430, 50346, 11955, 47539, 64207, 50275, 50129, 66753, 57973, 63312,
        51561, 38922,  6596, 13879, 56008, 14672, 15020, 63104, 35917, 35391])

In [14]:
sampler.indices[10:30]

tensor([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
        28, 29])

In [16]:
len(sampler)

63101

In [9]:
len(class_image_paths)

67709

In [30]:
weights = torch.tensor([0, 10, 3, 0], dtype=torch.float) # create a tensor of weights
torch.multinomial(weights, 2)

tensor([2, 1])

In [31]:
torch.multinomial(weights, 4) # ERROR!

RuntimeError: invalid multinomial distribution (with replacement=False, not enough non-negative category to sample)

In [33]:
torch.multinomial(weights, 4, replacement=True)

tensor([1, 1, 1, 2])

In [ ]:
class_image_paths[1]

In [ ]:
len(train_loader)

In [ ]:
inputs, classes = next(iter(train_loader))

In [ ]:
len(test_loader)

In [ ]:
len(valid_loader)

In [ ]:
show_batch(train_loader,4)

In [6]:
from torch.utils.data import Dataset, DataLoader

In [7]:
from torch.utils.data.sampler import Sampler, WeightedRandomSampler

In [8]:
numDataPoints = 1000
data_dim = 5
bs = 100

# Create dummy data with class imbalance 9 to 1
data = torch.FloatTensor(numDataPoints, data_dim)
target = np.hstack((np.zeros(int(numDataPoints * 0.9), dtype=np.int32),
                    np.ones(int(numDataPoints * 0.1), dtype=np.int32)))

print('target train 0/1: {}/{}'.format(
    len(np.where(target == 0)[0]), len(np.where(target == 1)[0])))

class_sample_count = np.array(
    [len(np.where(target == t)[0]) for t in np.unique(target)])




target train 0/1: 900/100


In [10]:
data.shape

torch.Size([1000, 5])

In [11]:
target.shape

(1000,)

In [12]:
class_sample_count

array([900, 100])

In [13]:
weight = 1. / class_sample_count

In [14]:
weight

array([0.00111111, 0.01      ])

In [15]:
samples_weight = np.array([weight[t] for t in target])

samples_weight = torch.from_numpy(samples_weight)
samples_weigth = samples_weight.double()


In [16]:
samples_weight

tensor([0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 

In [17]:
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

target = torch.from_numpy(target).long()
train_dataset = torch.utils.data.TensorDataset(data, target)

train_loader = DataLoader(
    train_dataset, batch_size=bs, num_workers=1, sampler=sampler)


In [18]:
for i, (data, target) in enumerate(train_loader):
    print("batch index {}, 0/1: {}/{}".format(
        i,
        len(np.where(target.numpy() == 0)[0]),
        len(np.where(target.numpy() == 1)[0])))

batch index 0, 0/1: 46/54
batch index 1, 0/1: 48/52
batch index 2, 0/1: 53/47
batch index 3, 0/1: 57/43
batch index 4, 0/1: 50/50
batch index 5, 0/1: 53/47
batch index 6, 0/1: 42/58
batch index 7, 0/1: 48/52
batch index 8, 0/1: 45/55
batch index 9, 0/1: 54/46
